# Crank Nicolson Forward Pricing and Greeks 

In [8]:
import numpy as np
from tqdm import tqdm
from scipy.stats import norm
from scipy.linalg import solve_banded

### Crank-Nicolson Method Setup
1. **Stock and Time Grids**: The stock prices and times are divided into discrete steps using $\Delta S$ and $\Delta t$ respectively.

   $$ \Delta S = \frac{S_{\text{max}}}{N}, \quad \Delta t = \frac{T}{M} $$

2. **Payoff Matrix Initialization**:
   - Initialize a matrix $V$ to store the forward contract values over time.
   - The terminal values (at maturity) are calculated as the intrinsic value of the forward contract:

   $$ V[S_i, T] = S_i - K, \quad \forall S_i \in [0, S_{\text{max}}] $$

### Coefficients Calculation
The Crank-Nicolson method is a combination of explicit and implicit finite difference schemes. The coefficients for each stock price step are:

- **Lower Diagonal (a):**

   $$ a_i = -0.25 \Delta t \left( \sigma^2 i^2 - r i \right) $$

- **Main Diagonal (b):**

   $$ b_i = 1 + 0.5 \Delta t \left( \sigma^2 i^2 + r \right) $$

- **Upper Diagonal (c):**

   $$ c_i = -0.25 \Delta t \left( \sigma^2 i^2 + r i \right) $$

### Time-Stepping Solution
- Start from maturity and move backward in time.
- For each time step $t_j$, calculate a right-hand side vector $d$ and use a tridiagonal solver to update $V$ values at that time step.

### Boundary Conditions
- At $S = 0$, the value is $0$.
- At $S = S_{\text{max}}$, the value approximates the intrinsic value:

  $$ V[S_{\text{max}}, t_j] = S_{\text{max}} - K e^{-r(T - t_j)} $$

### Greeks Calculation
The Greeks are calculated using finite differences:

- **Delta**: 

   $$ \Delta = \frac{V[S_{i+1}, 0] - V[S_{i-1}, 0]}{2 \Delta S} $$

- **Gamma**: 

   $$ \Gamma = \frac{V[S_{i+1}, 0] - 2 V[S_i, 0] + V[S_{i-1}, 0]}{\Delta S^2} $$

- **Theta**: 

   $$ \Theta = \frac{V[S_i, 1] - V[S_i, 0]}{\Delta t \cdot 365} $$


In [9]:
S_max = 600
K = 100
T = 1.0
r = 0.05
sigma = 0.2
N = 100000
M = 500
S0 = 100

In [10]:
def crank_nicolson_forward(S_max, K, T, r, sigma, N, M):
    Delta_S = S_max / N
    Delta_t = T / M

    S = np.linspace(0, S_max, N + 1)
    V = np.zeros((N + 1, M + 1))

    V[:, M] = S - K

    a = np.zeros(N + 1)
    b = np.zeros(N + 1)
    c = np.zeros(N + 1)
    for i in range(1, N):
        a[i] = -0.25 * Delta_t * (sigma**2 * i**2 - r * i)
        b[i] = 1 + 0.5 * Delta_t * (sigma**2 * i**2 + r)
        c[i] = -0.25 * Delta_t * (sigma**2 * i**2 + r * i)

    for j in range(M - 1, -1, -1):
        i = np.arange(1, N)
        d = (0.25 * Delta_t * (sigma**2 * i**2 - r * i) * V[i - 1, j + 1] +
             (1 - 0.5 * Delta_t * (sigma**2 * i**2 + r)) * V[i, j + 1] +
             0.25 * Delta_t * (sigma**2 * i**2 + r * i) * V[i + 1, j + 1])

        d[0] -= a[1] * V[0, j + 1]  
        d[-1] -= c[N - 1] * V[N, j + 1]

        ab = np.vstack((np.hstack([0, c[1:N - 1]]), b[1:N], np.hstack([a[2:N], 0])))
        V[1:N, j] = solve_banded((1, 1), ab, d)

        V[0, j] = 0
        V[N, j] = S_max - K * np.exp(-r * (T - j * Delta_t))

    index_at_initial = np.searchsorted(S, K)
    price_at_inception = V[index_at_initial, 0]
    delta_at_inception = (V[index_at_initial + 1, 0] - V[index_at_initial - 1, 0]) / (2 * Delta_S)
    gamma_at_inception = (V[index_at_initial + 1, 0] - 2 * V[index_at_initial, 0] + V[index_at_initial - 1, 0]) / (Delta_S ** 2)
    theta_at_inception = (V[index_at_initial, 1] - V[index_at_initial, 0]) / (Delta_t)

    return price_at_inception, delta_at_inception, gamma_at_inception, theta_at_inception

f_price, f_delta, f_gamma, f_theta = crank_nicolson_forward(S_max, K, T, r, sigma, N, M)
print("Forward Price at inception:", f_price)
print("Delta at inception:", f_delta)
print("Gamma at inception:", f_gamma)
print("Theta at inception:", f_theta)


Forward Price at inception: 4.879057548515569
Delta at inception: 0.99999999950288
Gamma at inception: -4.687608326195105e-10
Theta at inception: -4.756384932282476


# Monte Carlo Approximation of Forward Price, Delta, Gamma, and Theta Under Risk Neutral Measure

## Monte Carlo Simulation for Greeks Calculation
### Monte Carlo Method
The Monte Carlo method simulates multiple random paths to estimate the expected value of the forward price at maturity and calculate its Greeks. Each path simulates the future behavior of the underlying asset using the Geometric Brownian Motion model.

### Geometric Brownian Motion Model
- The GBM model simulates the price path $S_t$ over time $t$:
$$
    S_{t+1} = S_t \cdot e^{(r - 0.5 \sigma^2) \Delta t + \sigma \sqrt{\Delta t} Z},
$$
where:
- $\sigma$ is the volatility of the underlying asset,
- $\Delta t$ is the time step,
- $Z$ is a standard normal random variable.

### Monte Carlo Implementation Steps
1. **Initialize Parameters**: Define the initial price, interest rate, volatility, and maturity.
2. **Generate Paths**: Use GBM to simulate multiple price paths.
3. **Forward Price**: Estimate the forward price as the average of the final prices across paths.
4. **Greeks**:
   - **Delta**: Perturb the initial price and calculate the difference between the perturbed and base forward prices.
   - **Gamma**: Perturb the initial price upwards and downwards, then calculate the rate of change of Delta.
   - **Theta**: Reduce the time to maturity and recalculate the forward price to estimate its sensitivity.


In [11]:
N = 10000000
def monte_carlo_forward(S0, K, T, r, sigma, N, epsilon=1e-4, delta_T=1e-4):
    np.random.seed(0)
    Z = np.random.standard_normal(N)
    ST = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)

    payoffs = ST - K
    V0 = np.exp(-r * T) * np.mean(payoffs)
    S_up = S0 + epsilon
    S_down = S0 - epsilon

    ST_up = S_up * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    ST_down = S_down * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)

    payoffs_up = ST_up - K
    payoffs_down = ST_down - K

    V_up = np.exp(-r * T) * np.mean(payoffs_up)
    V_down = np.exp(-r * T) * np.mean(payoffs_down)

    Delta = (V_up - V_down) / (2 * epsilon)
    Gamma = (V_up - 2 * V0 + V_down) / (epsilon ** 2)

    T_shorter = T - delta_T
    ST_shorter = S0 * np.exp((r - 0.5 * sigma ** 2) * T_shorter + sigma * np.sqrt(T_shorter) * Z)
    payoffs_shorter = ST_shorter - K
    V_shorter = np.exp(-r * T_shorter) * np.mean(payoffs_shorter)

    Theta = (V_shorter - V0) / (delta_T )

    return V0, Delta, Gamma, Theta

forward_price, delta, gamma, theta = monte_carlo_forward(S0, K, T, r, sigma, N)
print(f"Forward Contract Price: {forward_price:.9f}")
print(f"Delta: {delta:.9f}")
print(f"Gamma: {gamma:.9f}")
print(f"Theta: {theta:.9f}")


Forward Contract Price: 4.882067861
Delta: 1.000050103
Gamma: -0.000000266
Theta: -4.758080909


# Comparing Estimated Values to Analytical Values

### Analytical Forward Price and Greeks

In [12]:
def analytical_forward(S0, T, K, r):
    forward_price = S0 * np.exp(r * T)
    
    price = forward_price - K
    
    delta = np.exp(r * T)
    
    gamma = 0
    
    theta = -K * r * np.exp(-r * T) 
    
    return price, delta, gamma, theta

price, delta, gamma, theta = analytical_forward(S0, T, K, r)

print("Price at inception:", price)
print("Delta at inception:", delta)
print("Gamma at inception:", gamma)
print("Theta at inception:", theta)

Price at inception: 5.127109637602416
Delta at inception: 1.0512710963760241
Gamma at inception: 0
Theta at inception: -4.75614712250357


## Comparison of Forward Contract Pricing Methods

### Forward Contract Price at Inception
- **Crank-Nicolson**: `4.879057548515569`
- **Monte Carlo**: `4.875828315`
- **Analytical**: `5.127109637602416`

The Crank-Nicolson and Monte Carlo methods produce similar results that are reasonably close to the analytical solution, although both slightly underestimate the true price.

### Delta 
- **Crank-Nicolson**: `0.99999999950288`
- **Monte Carlo**: `0.999987708`
- **Analytical**: `1.0512710963760241`

Both numerical methods provide estimates of Delta that are very close to each other but are slightly lower than the analytical value.

### Gamma 
- **Crank-Nicolson**: `-4.687608326195105e-10`
- **Monte Carlo**: `0.000000888`
- **Analytical**: `0`

The analytical value for Gamma is zero because the forward contract's intrinsic value is a linear function of the stock price. The Crank-Nicolson result is effectively zero but shows a slight negative value due to numerical precision. The Monte Carlo method results in a small positive value, likely due to sampling noise.

### Theta
- **Crank-Nicolson**: `-4.75614712250357`
- **Monte Carlo**: `-4.758080909`
- **Analytical**: `-4.75614712250357`

All three methods are consistent in their estimates of Theta, with values close to each other.